### Import Libraries


In [45]:
import random
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import seaborn as sns
import pandas as pd 

BOARD_SIZE = 100 

### Board Generation


In [46]:
def create_snakes_and_ladders(board_size, num_snakes, num_ladders):
    """
    Generates a board with randomised start and end points for snakes and ladders.
    """
    snakes = {} 
    ladders = {}
    used_tiles = set() 
    
    def generate_endpoints(entity_type, used_tiles):
        # ... (generate_endpoints function - same as before)
        start_tile = 0
        end_tile = 0
        while True:
            if entity_type == "snake":
                start_tile = random.randint(2, board_size - 1)
            elif entity_type == "ladder":
                start_tile = random.randint(2, board_size - 3)

            if start_tile in used_tiles:
                continue

            if entity_type == "snake":
                end_tile = random.randint(1, start_tile - 1)
            elif entity_type == "ladder":
                end_tile = random.randint(start_tile + 1, board_size - 1)

            if end_tile in used_tiles or end_tile == start_tile:
                continue

            valid_placement = True
            for s_start, s_end in snakes.items():
                if start_tile == s_start or start_tile == s_end or end_tile == s_start or end_tile == s_end:
                    valid_placement = False
                    break
            if not valid_placement: continue
            for l_start, l_end in ladders.items():
                if start_tile == l_start or start_tile == l_end or end_tile == l_start or end_tile == l_end:
                    valid_placement = False
                    break
            if not valid_placement: continue

            break

        used_tiles.add(start_tile)
        used_tiles.add(end_tile)
        return start_tile, end_tile

    for _ in range(num_snakes):
        start, end = generate_endpoints("snake", used_tiles) 
        snakes[start] = end

    for _ in range(num_ladders):
        start, end = generate_endpoints("ladder", used_tiles) 
        ladders[start] = end

    return snakes, ladders

### Game Simulation


In [47]:
def simulate_game(board_size, snakes, ladders, record_positions=False, record_entity_triggers=False):
    """
    Simulates a single game of Snakes and Ladders and returns metrics.
    ... (rest of the docstring is the same)
    """
    position = 0
    turns = 0
    positions_visited = [] if record_positions else None
    entity_triggers = [] if record_entity_triggers else None

    if record_positions:
        positions_visited.append(position)

    while position < board_size:
        roll = random.randint(1, 6)
        position += roll

        if position > board_size:
            position -= roll
            continue

        turns += 1
        if record_positions:
            positions_visited.append(position)

        if position in snakes:
            position = snakes[position]
            if record_positions:
                positions_visited.append(position)
        elif position in ladders:
            position = ladders[position]
            if record_positions:
                positions_visited.append(position)

    return turns, positions_visited, entity_triggers


def run_simulations(num_simulations, board_size, num_snakes, num_ladders, record_positions=False, record_entity_triggers=False):
    """
    Runs multiple simulations and collects metrics, turn counts, positions, and entity triggers.
    ... (rest of the docstring is the same)
    """
    turn_counts = []
    all_positions_visited = [] if record_positions else None
    all_entity_triggers = [] if record_entity_triggers else None
    last_snakes_config = {}
    last_ladders_config = {}

    for _ in range(num_simulations):
        snakes_config, ladders_config = create_snakes_and_ladders(board_size, num_snakes, num_ladders)
        turns, positions_visited, entity_triggers = simulate_game(board_size, snakes_config, ladders_config, record_positions, record_entity_triggers)
        turn_counts.append(turns)
        if record_positions:
            all_positions_visited.append(positions_visited)
        if record_entity_triggers:
            all_entity_triggers.append(entity_triggers)
        last_snakes_config = snakes_config
        last_ladders_config = ladders_config

    average_turns = sum(turn_counts) / num_simulations
    turn_distribution = Counter(turn_counts)

    metrics = {
        "average_turns": average_turns,
        "turn_distribution": turn_distribution,
        "all_turn_counts": turn_counts
    }
    return metrics, turn_counts, all_positions_visited, last_snakes_config, last_ladders_config, all_entity_triggers

### Markov Modelling


In [48]:
def create_transition_matrix(board_size, snakes, ladders):
    """
    Creates a transition matrix for the Snakes and Ladders Markov model.

    Args:
        board_size (int): Total number of tiles.
        snakes (dict): Snakes configuration (start: end).
        ladders (dict): Ladders configuration (start: end).

    Returns:
        pandas.DataFrame: Transition matrix where rows and columns represent tiles (0 to board_size).
    """
    num_states = board_size + 1 # States 0 to 100 (including start and finish)
    transition_matrix = pd.DataFrame(0.0, index=range(num_states), columns=range(num_states))

    for current_tile in range(num_states):
        if current_tile == board_size: # Terminal state
            transition_matrix.loc[current_tile, current_tile] = 1.0 # Stays at 100
            continue

        for roll in range(1, 7):
            next_position = current_tile + roll
            if next_position > board_size: # Overshooting
                next_position = current_tile # Stay in place

            if next_position in snakes:
                next_position = snakes[next_position]
            elif next_position in ladders:
                next_position = ladders[next_position]

            transition_matrix.loc[current_tile, next_position] += 1/6.0 # Probability of each roll

    return transition_matrix

def calculate_expected_turns_markov(transition_matrix, board_size):
    """
    Calculates the expected number of turns to finish the game using the transition matrix.

    Args:
        transition_matrix (pandas.DataFrame): Transition matrix.
        board_size (int): Total number of tiles.

    Returns:
        float: Expected number of turns.
    """
    n = board_size # number of non-terminal states
    Q = transition_matrix.iloc[1:n+1, 1:n+1].copy() # Q matrix (non-absorbing to non-absorbing)
    I = pd.DataFrame(np.identity(n), index=range(1, n+1), columns=range(1, n+1)) # Identity matrix
    N_inv = np.linalg.inv(I - Q) # Fundamental matrix (inverse of (I-Q))
    N = pd.DataFrame(N_inv, index=range(1, n+1), columns=range(1, n+1))
    ones_vector = pd.DataFrame(np.ones(n), index=range(1, n+1), columns=['ones']) # Vector of ones
    expected_turns_df = N.dot(ones_vector) # Expected turns vector

    return expected_turns_df.loc[1, 'ones'] # Expected turns from starting position 1


def calculate_steady_state_distribution(transition_matrix, board_size, initial_state=0, num_iterations=500):
    """
    Calculates the steady-state distribution of tile occupation probabilities.

    Args:
        transition_matrix (pandas.DataFrame): Transition matrix.
        board_size (int): Total number of tiles.
        initial_state (int): Starting state for iteration (default 0).
        num_iterations (int): Number of iterations to run.

    Returns:
        pandas.Series: Steady-state distribution (probability of being on each tile).
    """
    num_states = board_size + 1
    state_distribution = pd.Series([0.0] * num_states, index=range(num_states))
    state_distribution[initial_state] = 1.0 # Start at state 0

    for _ in range(num_iterations): # Iterate for convergence
        state_distribution = state_distribution.dot(transition_matrix)

    return state_distribution


### Plotting


In [49]:
def plot_turn_distribution(turn_counts, num_snakes, num_ladders, num_simulations):
    # ... (plot_turn_distribution function - same as before)
    plt.figure(figsize=(10, 6))
    plt.hist(turn_counts, bins=range(min(turn_counts), max(turn_counts) + 2), align='left', rwidth=0.8, color='skyblue', edgecolor='black', density=True)
    plt.xlabel("Number of Turns")
    plt.ylabel("Probability")
    plt.title(f"Probability Distribution of Turns to Complete Snakes and Ladders\n({num_snakes} Snakes, {num_ladders} Ladders, {num_simulations} Simulations)")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(range(min(turn_counts), max(turn_counts) + 1, 5))
    plt.tight_layout()
    plt.show()

def plot_cdf(turn_counts, num_snakes, num_ladders, num_simulations):
    # ... (plot_cdf function - same as before)
    plt.figure(figsize=(10, 6))
    sorted_turns = np.sort(turn_counts)
    cdf_values = np.arange(1, len(sorted_turns) + 1) / len(sorted_turns)
    plt.plot(sorted_turns, cdf_values, marker='.', linestyle='-', color='coral')
    plt.xlabel("Number of Turns")
    plt.ylabel("Cumulative Probability")
    plt.title(f"Cumulative Distribution Function (CDF) of Turns\n({num_snakes} Snakes, {num_ladders} Ladders, {num_simulations} Simulations)")
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

def plot_board_heatmap(all_positions_visited, board_size, snakes, ladders, num_simulations):
    # ... (plot_board_heatmap function - same as before)
    tile_counts = Counter()
    for positions_list in all_positions_visited:
        tile_counts.update(positions_list)

    board_grid = np.zeros((10, 10))
    for tile in range(1, board_size + 1):
        row = (tile - 1) // 10
        col = (tile - 1) % 10
        if row % 2 == 1:
            col = 9 - col
        board_grid[9 - row, col] = tile_counts.get(tile, 0)

    plt.figure(figsize=(12, 10))
    sns.heatmap(board_grid, annot=False, fmt="d", cmap="viridis", linewidths=.5, cbar_kws={'label': 'Visit Frequency'})
    plt.title(f"Tile Visit Frequency Heatmap ({num_simulations} Simulations)\nSnakes={len(snakes)}, Ladders={len(ladders)}")
    plt.xticks([])
    plt.yticks([])

    for start, end in snakes.items():
        plt.text( ((start-1)%10) if ((start-1)//10)%2 == 0 else (9-((start-1)%10)), 9-((start-1)//10), 'S', color='white', ha='center', va='center', fontsize=8)
    for start, end in ladders.items():
        plt.text( ((start-1)%10) if ((start-1)//10)%2 == 0 else (9-((start-1)%10)), 9-((start-1)//10), 'L', color='white', ha='center', va='center', fontsize=8)

    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

def plot_combined_entity_trigger_heatmap(all_entity_triggers, board_size, num_simulations, snakes, ladders):
    # ... (plot_combined_entity_trigger_heatmap function - same as before)
    snake_trigger_counts = Counter()
    ladder_trigger_counts = Counter()

    snake_start_tiles = set(snakes.keys())
    ladder_start_tiles = set(ladders.keys())

    for triggers_list in all_entity_triggers:
        for tile, entity_type in triggers_list:
            if entity_type == 'snake':
                snake_trigger_counts[tile] += 1
            elif entity_type == 'ladder':
                ladder_trigger_counts[tile] += 1

    snake_board_grid = np.zeros((10, 10))
    ladder_board_grid = np.zeros((10, 10))

    for tile in range(1, board_size + 1):
        row = (tile - 1) // 10
        col = (tile - 1) % 10
        if row % 2 == 1:
            col = 9 - col

        snake_board_grid[9 - row, col] = snake_trigger_counts.get(tile, 0)
        ladder_board_grid[9 - row, col] = ladder_trigger_counts.get(tile, 0)

    combined_grid = snake_board_grid + ladder_board_grid

    cmap = sns.color_palette(['lightgray', 'red', 'green'])
    norm = plt.Normalize(vmin=0, vmax=combined_grid.max() if combined_grid.max() > 0 else 2)

    annot_grid = np.full_like(combined_grid, '', dtype=object)
    for tile in range(1, board_size + 1):
        if tile in snake_start_tiles or tile in ladder_start_tiles:
            row = (tile - 1) // 10
            col = (tile - 1) % 10
            if row % 2 == 1:
                col = 9 - col
            val = 0
            if snake_board_grid[9 - row, col] > 0: val += 1
            if ladder_board_grid[9 - row, col] > 0: val += 2

            if val == 1: annot_grid[9-row, col] = 'S'
            elif val == 2: annot_grid[9-row, col] = 'L'
            elif val == 3: annot_grid[9-row, col] = 'SL'

    plt.figure(figsize=(12, 10))
    sns.heatmap(combined_grid, annot=annot_grid, fmt="", cmap=cmap, linewidths=.5, cbar=True, norm=norm)
    plt.title(f"Combined Snake Head & Ladder Base Trigger Intensity Heatmap ({num_simulations} Simulations)")
    plt.xticks([])
    plt.yticks([])
    cbar = plt.gca().collections[0].colorbar
    cbar.set_label('Trigger Count')

    if snakes:
        for start, end in snakes.items():
            plt.text( ((start-1)%10) if ((start-1)//10)%2 == 0 else (9-((start-1)%10)), 9-((start-1)//10), 'S', color='white', ha='center', va='center', fontsize=8)
    if ladders:
        for start, end in ladders.items():
            plt.text( ((start-1)%10) if ((start-1)//10)%2 == 0 else (9-((start-1)%10)), 9-((start-1)//10), 'L', color='white', ha='center', va='center', fontsize=8)

    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

### Function Call


In [50]:
# Run simulations
board_size = 100
num_snakes = 8
num_ladders = 8
num_simulations = 10000
simulation_metrics, turn_counts, all_positions_visited, snakes_config, ladders_config, all_entity_triggers = run_simulations(num_simulations, board_size, num_snakes, num_ladders, record_positions=True, record_entity_triggers=True)

print("\nSimulation Metrics:")
print(f"Average Turns (Simulation): {simulation_metrics['average_turns']:.2f}")
print("Turn Distribution (first 20):", dict(list(simulation_metrics['turn_distribution'].items())[:20]))

# Markov Model Analysis
transition_matrix = create_transition_matrix(board_size, snakes_config, ladders_config)
expected_turns_markov = calculate_expected_turns_markov(transition_matrix, board_size)
steady_state_distribution = calculate_steady_state_distribution(transition_matrix, board_size)

print(f"\nMarkov Model Metrics:")
print(f"Expected Turns (Markov Model): {expected_turns_markov:.2f}")
print("\nSteady-State Distribution (Top 20 Tiles):")
print(steady_state_distribution.sort_values(ascending=False).head(20))


# Generate and display visualizations
plot_turn_distribution(turn_counts, num_snakes, num_ladders, num_simulations)
plot_cdf(turn_counts, num_snakes, num_ladders, num_simulations)
plot_board_heatmap(all_positions_visited, board_size, snakes_config, ladders_config, num_simulations)
plot_combined_entity_trigger_heatmap(all_entity_triggers, board_size, num_simulations, snakes=snakes_config, ladders=ladders_config)

# Optional: Visualize Steady-State Distribution as Heatmap
plt.figure(figsize=(12, 10))
board_grid_steady_state = np.zeros((10, 10))
for tile in range(1, board_size + 1):
    row = (tile - 1) // 10
    col = (tile - 1) % 10
    if row % 2 == 1:
        col = 9 - col
    board_grid_steady_state[9 - row, col] = steady_state_distribution[tile]

sns.heatmap(board_grid_steady_state, cmap="viridis", linewidths=.5, cbar_kws={'label': 'Steady-State Probability'})
plt.title(f"Steady-State Tile Occupation Probability Heatmap")
plt.xticks([])
plt.yticks([])
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


Simulation Metrics:
Average Turns (Simulation): 31.88
Turn Distribution (first 20): {15: 302, 33: 160, 29: 212, 11: 228, 47: 68, 12: 244, 37: 130, 78: 22, 28: 249, 38: 130, 18: 285, 27: 209, 7: 153, 53: 55, 109: 6, 16: 256, 61: 34, 35: 169, 113: 7, 25: 287}


LinAlgError: Singular matrix

The fact that you can model the game effectively with a Markov model reinforces the idea that Snakes and Ladders is indeed a memoryless process.
<br>The next move only depends on the current tile and the dice roll, not on the history of previous moves.
